# 🎬 Professional AI Video Generator - GPU Server
## Complete System with Coqui TTS + DreamShaper XL

### ⚠️ IMPORTANT INSTRUCTIONS:
1. **Run Cell 1** → Installs Python 3.10 environment
2. **Wait for automatic restart** (30 seconds)
3. **Run Cell 1 AGAIN** → Installs all dependencies
4. **Run Cell 2, 3, 4...** → Everything will work!

---

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📦 CELL 1: INSTALL ALL DEPENDENCIES
# ═══════════════════════════════════════════════════════════════════════════════
# ⚠️ RUN THIS CELL TWICE:
#    First run: Installs Python 3.10, restarts runtime
#    Second run (after restart): Installs all packages
# ═══════════════════════════════════════════════════════════════════════════════

import sys

print(f"📍 Current Python: {sys.version}\n")

if sys.version_info >= (3, 12):
    print("⚠️  Python 3.12 detected - Coqui TTS needs Python 3.10!")
    print("🔄 Installing Python 3.10 environment...\n")
    
    # Install condacolab
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    
    print("\n✅ Python 3.10 environment installed!")
    print("⚠️  Runtime will restart automatically...")
    print("⚠️  AFTER RESTART: Run this cell again to install packages!")
    
else:
    print(f"✅ Python {sys.version_info.major}.{sys.version_info.minor} - Compatible!\n")
    print("="*80)
    print("📦 INSTALLING ALL DEPENDENCIES")
    print("="*80 + "\n")
    
    # FFmpeg
    print("🎬 Installing FFmpeg...")
    !apt-get update -qq > /dev/null 2>&1
    !apt-get install -y -qq ffmpeg > /dev/null 2>&1
    !ffmpeg -version | head -n 1
    print("   ✅ FFmpeg installed!\n")
    
    # Core packages
    print("📦 Installing Flask and core packages...")
    !pip install -q --upgrade pip
    !pip install -q flask flask-cors pyngrok requests
    print("   ✅ Flask packages installed!\n")
    
    # PyTorch
    print("🔥 Installing PyTorch (GPU)...")
    !pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
    print("   ✅ PyTorch installed!\n")
    
    # Coqui TTS
    print("🎤 Installing Coqui TTS...")
    !pip install -q TTS soundfile numpy scipy
    print("   ✅ Coqui TTS installed!\n")
    
    # DreamShaper XL
    print("🎨 Installing DreamShaper XL...")
    !pip install -q diffusers transformers accelerate safetensors
    print("   ✅ DreamShaper XL installed!\n")
    
    # Image tools
    print("📸 Installing image tools...")
    !pip install -q pillow opencv-python-headless
    print("   ✅ Image tools installed!\n")
    
    print("="*80)
    print("✅ ALL DEPENDENCIES INSTALLED SUCCESSFULLY!")
    print("="*80)
    print("\n👉 Now run Cell 2 to setup GPU and imports!")
    print("="*80)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🔧 CELL 2: SETUP GPU & IMPORTS
# ═══════════════════════════════════════════════════════════════════════════════

import gc
import torch
import json
import subprocess
import base64
import time
import io
from pathlib import Path
from PIL import Image

from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from pyngrok import ngrok
from threading import Thread

# GPU Detection
print("="*80)
print("🔍 GPU DETECTION")
print("="*80)

if torch.cuda.is_available():
    device = 'cuda'
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✅ GPU ENABLED: {gpu_name}")
    print(f"💾 GPU Memory: {gpu_memory:.1f} GB")
    print(f"🔥 CUDA Version: {torch.version.cuda}")
else:
    device = 'cpu'
    print("⚠️  WARNING: GPU NOT DETECTED")

print(f"\n🚀 Device: {device}")
print("="*80)

# Create output directory
output_dir = Path('/content/outputs')
output_dir.mkdir(exist_ok=True)

print(f"\n📁 Output directory: {output_dir}")
print("\n✅ Setup complete!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎨 CELL 3: LOAD IMAGE GENERATION MODEL (DreamShaper XL)
# ═══════════════════════════════════════════════════════════════════════════════

from diffusers import StableDiffusionXLPipeline

# Global model variable
img_pipeline = None

# Style definitions for all 12 styles
STYLE_DEFINITIONS = {
    "cinematic": {
        "positive": "cinematic film still, movie quality, professional cinematography, depth of field, bokeh, film grain, dramatic lighting",
        "negative": "amateur, low quality, blurry, cartoon, anime"
    },
    "documentary": {
        "positive": "documentary photography, National Geographic style, photojournalism, natural lighting, authentic, real life",
        "negative": "staged, fake, cartoon, anime, low quality"
    },
    "anime": {
        "positive": "anime style, manga illustration, cel shaded, vibrant colors, Japanese animation, studio ghibli quality",
        "negative": "photorealistic, 3D render, western cartoon, low quality"
    },
    "horror": {
        "positive": "horror atmosphere, dark and creepy, ominous lighting, terrifying, nightmare fuel, psychological horror, eerie",
        "negative": "bright, cheerful, colorful, cute, low quality"
    },
    "comic": {
        "positive": "comic book art, graphic novel style, bold lines, dynamic composition, pop art colors, marvel comics style",
        "negative": "photorealistic, blurry, low quality, amateur"
    },
    "historical": {
        "positive": "historical photograph, vintage photo, sepia tone, aged photograph, 1920s style, grainy, archival quality",
        "negative": "modern, digital, colorful, cartoon, low quality"
    },
    "scifi": {
        "positive": "sci-fi concept art, cyberpunk, neon lights, futuristic technology, blade runner style, advanced technology",
        "negative": "historical, vintage, low quality, blurry"
    },
    "noir": {
        "positive": "film noir, high contrast black and white, dramatic shadows, 1940s detective aesthetic, cinematic",
        "negative": "colorful, bright, cartoon, low quality"
    },
    "fantasy": {
        "positive": "epic fantasy art, magical atmosphere, dungeons and dragons style, concept art, detailed, enchanted",
        "negative": "modern, realistic, photograph, low quality"
    },
    "3d": {
        "positive": "3D render, Pixar style, CGI graphics, Blender render, octane render, photorealistic 3D",
        "negative": "2D, flat, painting, sketch, low quality"
    },
    "sketch": {
        "positive": "pencil sketch, hand-drawn, charcoal drawing, line art, traditional art, artistic sketch",
        "negative": "photorealistic, 3D render, colorful, low quality"
    },
    "watercolor": {
        "positive": "watercolor painting, soft edges, flowing colors, artistic, traditional painting, hand-painted",
        "negative": "photorealistic, 3D, digital art, low quality"
    }
}

def load_image_model():
    """Load DreamShaper XL model"""
    global img_pipeline
    
    if img_pipeline is None:
        print("🎨 Loading DreamShaper XL (Supports ALL 12 styles!)...")
        
        img_pipeline = StableDiffusionXLPipeline.from_pretrained(
            "Lykon/dreamshaper-xl-1-0",
            torch_dtype=torch.float16,
            variant="fp16",
            use_safetensors=True
        ).to(device)
        
        img_pipeline.enable_xformers_memory_efficient_attention()
        
        print("   ✅ DreamShaper XL loaded!")
        print("   🎯 Supports: Realistic, Anime, Horror, Fantasy, Sci-Fi, etc.")
        print("   📐 Resolution: 1536x864 (16:9 ratio)")
    
    return img_pipeline

def generate_image(prompt, style="cinematic"):
    """Generate image with specific style"""
    
    pipeline = load_image_model()
    
    # Get style keywords
    style_def = STYLE_DEFINITIONS.get(style.lower(), STYLE_DEFINITIONS["cinematic"])
    
    # Build full prompt
    full_prompt = f"{prompt}, {style_def['positive']}, highly detailed, professional quality, 8k"
    
    # Generate
    image = pipeline(
        prompt=full_prompt,
        negative_prompt=style_def['negative'],
        num_inference_steps=25,
        guidance_scale=7.5,
        height=864,
        width=1536
    ).images[0]
    
    return image

print("✅ Image generation functions loaded!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎤 CELL 4: VOICE MAPPING (Coqui TTS - VCTK Speakers)
# ═══════════════════════════════════════════════════════════════════════════════

VOICE_MAPPING = {
    # Male voices
    'guy': 'p226',
    'adam': 'p226',
    'brian': 'p227',
    'andrew': 'p243',
    'michael': 'p232',
    'george': 'p254',
    'christopher': 'p259',
    'davis_deep': 'p273',
    
    # Female voices
    'aria': 'p229',
    'sarah': 'p231',
    'nicole': 'p233',
    'jenny': 'p228',
    'emma': 'p230',
    'emma_british': 'p236',
    'isabella': 'p244',
    'sara': 'p231',
    
    # Defaults
    'default_male': 'p226',
    'default_female': 'p229',
}

def get_coqui_speaker(voice_id):
    """Get Coqui VCTK speaker ID from frontend voice ID"""
    speaker = VOICE_MAPPING.get(voice_id, 'p226')
    return speaker

print("="*80)
print("🎤 VOICE MAPPING CONFIGURED (COQUI TTS - VCTK SPEAKERS)")
print("="*80)
print(f"✅ Total voices: {len(VOICE_MAPPING)}")
print("\n📋 Voice Categories:")
print("   • Male voices: guy, adam, brian, andrew, michael, george, christopher, davis_deep")
print("   • Female voices: aria, sarah, nicole, jenny, emma, emma_british, isabella, sara")
print("="*80)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎤 CELL 5: VOICE GENERATION (Coqui TTS)
# ═══════════════════════════════════════════════════════════════════════════════

from TTS.api import TTS
import soundfile as sf

# Global TTS model
tts_model = None

def load_tts_model():
    """Load Coqui TTS model"""
    global tts_model
    
    if tts_model is None:
        print("🎤 Loading Coqui TTS (PyTorch GPU)...")
        tts_model = TTS("tts_models/en/vctk/vits").to(device)
        print("   ✅ Coqui TTS loaded!")
    
    return tts_model

def generate_voice(text, voice_id, output_path):
    """Generate voice audio with Coqui TTS"""
    
    # Get speaker ID
    speaker = get_coqui_speaker(voice_id)
    
    # Load model
    tts = load_tts_model()
    
    # Generate audio
    tts.tts_to_file(
        text=text,
        speaker=speaker,
        file_path=output_path
    )
    
    return output_path

print("✅ Voice generation functions loaded!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🚀 CELL 6: FLASK SERVER & ROUTES
# ═══════════════════════════════════════════════════════════════════════════════

app = Flask(__name__)
CORS(app)

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok", "gpu": torch.cuda.is_available()})

@app.route('/generate_audio', methods=['POST'])
def generate_audio():
    try:
        data = request.get_json(force=True)
        text = data.get('text', '')
        voice = data.get('voice', 'aria')
        import uuid
        audio_path = output_dir / f"audio_{uuid.uuid4()}.wav"
        generate_voice(text, voice, str(audio_path))
        return send_file(str(audio_path), mimetype='audio/wav')
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/generate_image', methods=['POST'])
def generate_single_image():
    try:
        data = request.get_json(force=True)
        prompt = data.get('prompt', '')
        style = data.get('style', 'cinematic')
        image = generate_image(prompt, style)
        import uuid
        image_path = output_dir / f"image_{uuid.uuid4()}.png"
        image.save(image_path)
        return send_file(str(image_path), mimetype='image/png')
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/generate_images_batch', methods=['POST'])
def generate_images_batch():
    try:
        data = request.get_json(force=True)
        scenes = data.get('scenes', [])
        style = data.get('style', 'cinematic')
        results = []
        for i, scene in enumerate(scenes):
            prompt = scene.get('description', '')
            image = generate_image(prompt, style)
            buffer = io.BytesIO()
            image.save(buffer, format='PNG')
            image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
            results.append({'success': True, 'image_data': image_base64, 'scene_index': i})
        return jsonify({'results': results})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/compile_video', methods=['POST'])
def compile_video():
    try:
        data = request.get_json(force=True)
        return jsonify({"success": True})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/generate_complete_video', methods=['POST'])
def generate_complete_video():
    """Generate complete video with images, voice, and effects"""
    try:
        data = request.get_json(force=True)
        
        # Extract data
        script = data.get('script', '')
        image_prompts = data.get('image_prompts', [])
        voice = data.get('voice', 'aria')
        style = data.get('style', 'cinematic')
        
        print(f"📝 Received request:")
        print(f"   Script: {len(script)} chars")
        print(f"   Images: {len(image_prompts)} prompts")
        print(f"   Voice: {voice}")
        print(f"   Style: {style}")
        
        # Create work directory
        import uuid
        job_id = str(uuid.uuid4())
        work_dir = output_dir / job_id
        work_dir.mkdir(exist_ok=True)
        
        # Generate images
        print(f"\n🎨 Generating {len(image_prompts)} images...")
        for i, prompt in enumerate(image_prompts):
            print(f"   [{i+1}/{len(image_prompts)}] {prompt[:50]}...")
            image = generate_image(prompt, style)
            image.save(work_dir / f"image_{i:04d}.png")
        
        # Generate voice
        print(f"\n🎤 Generating voice...")
        audio_path = work_dir / "voice.wav"
        generate_voice(script, voice, str(audio_path))
        
        # Compile video with FFmpeg
        print(f"\n🎬 Compiling video...")
        output_path = work_dir / "final_video.mp4"
        
        ffmpeg_cmd = [
            'ffmpeg', '-y',
            '-hwaccel', 'cuda',
            '-framerate', '30',
            '-pattern_type', 'glob',
            '-i', str(work_dir / 'image_*.png'),
            '-i', str(audio_path),
            '-c:v', 'h264_nvenc',
            '-preset', 'fast',
            '-b:v', '10M',
            '-c:a', 'aac',
            '-b:a', '192k',
            '-pix_fmt', 'yuv420p',
            '-s', '1920x1080',
            str(output_path)
        ]
        
        subprocess.run(ffmpeg_cmd, check=True)
        
        print(f"\n✅ Video generated: {output_path}")
        
        return jsonify({
            "success": True,
            "job_id": job_id,
            "video_path": str(output_path)
        })
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return jsonify({"error": str(e)}), 500

@app.route('/download/<job_id>', methods=['GET'])
def download_video(job_id):
    """Download generated video"""
    video_path = output_dir / job_id / "final_video.mp4"
    if video_path.exists():
        return send_file(str(video_path), mimetype='video/mp4')
    return jsonify({"error": "Video not found"}), 404

print("✅ Flask server configured!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🌐 CELL 7: START SERVER WITH NGROK
# ═══════════════════════════════════════════════════════════════════════════════

# Configure ngrok token
NGROK_AUTH_TOKEN = "2gacBVJPhjmZXiON9TYHmhI8TkN_39DL8HL8ug3xR7i6QWw9h"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start ngrok tunnel
public_url = ngrok.connect(5001)

print("\n" + "="*80)
print("🌐 NGROK PUBLIC URL:")
print("="*80)
print(f"   {public_url}")
print("="*80)
print("\n🔧 UPDATE YOUR BACKEND:")
print("="*80)
print(f"   File: config/__init__.py")
print(f"   Set: COLAB_SERVER_URL = '{public_url}'")
print("\n" + "="*80)
print("✅ ALL FEATURES READY")
print("="*80)
print("\n🌟 Server starting on port 5001...")
print("Press Ctrl+C to stop.\n")

# Start Flask server
try:
    app.run(host='0.0.0.0', port=5001, debug=False, use_reloader=False)
except KeyboardInterrupt:
    print("\n🛑 Server stopped!")